# load data that will be imported later

This hurricane data will be imported later

In [1]:
import pandas as pd 
data = pd.read_csv("https://dq-content.s3.amazonaws.com/251/storm_data.csv")

# Connect to database

 Before connecting to AWS RDS, set up the database first.

In [6]:
import psycopg2
conn = psycopg2.connect(
    database="<db-alias>",
    user="<username>",
    password="<password>",
    host="<host>",
    port='<port-usually-5432>'
)
cur = conn.cursor()

In [7]:
# create table
cur.execute("""
drop table if exists hurricane;
create table hurricane(
fid int primary key,
year smallint,
month smallint,
day smallint,
ad_time varchar(5),
btid smallint,
name text,
lat numeric(3,1),
long numeric(4,1),
wind_kts smallint,
pressure smallint,
cat varchar(2),
basin text,
shape_leng real);
""")
conn.commit()

In [10]:
# import the csv
for index,row in data.iterrows():
    cur.execute("insert into hurricane values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)",row)
conn.commit()

In [8]:
# create group who can only read the table
# create a role which can only read this table
cur.execute("""
create role viewer_role with nologin;
revoke all on hurricane from viewer_role;
grant insert, update, select on hurricane to viewer_role;
""")
conn.commit()

In [9]:
#assign jack to viewer role
cur.execute("""
create role jack with login password 'jackpass' in role viewer_role;
""")
conn.commit()

In [12]:
# close the connection
conn.close()

# Connect using the newly created readonly user

In [19]:
conn = psycopg2.connect(
    database="<db-alias>",
    user="jack",
    password="jackpass",
    host="<host>",
    port='5432'
)
cur = conn.cursor()

In [21]:
cur.execute("select * from hurricane limit 10;")
result = cur.fetchall()
for row in result:
    print(row)

(2001, 1957, 8, 8, '1800Z', 63, 'NOTNAMED', Decimal('22.5'), Decimal('-140.0'), 50, 0, 'TS', 'Eastern Pacific', 1.14017)
(2002, 1961, 10, 3, '1200Z', 116, 'PAULINE', Decimal('22.1'), Decimal('-140.2'), 45, 0, 'TS', 'Eastern Pacific', 1.16619)
(2003, 1962, 8, 29, '0600Z', 124, 'C', Decimal('18.0'), Decimal('-140.0'), 45, 0, 'TS', 'Eastern Pacific', 2.10238)
(2004, 1967, 7, 14, '0600Z', 168, 'DENISE', Decimal('16.6'), Decimal('-139.5'), 45, 0, 'TS', 'Eastern Pacific', 2.12132)
(2005, 1972, 8, 16, '1200Z', 251, 'DIANA', Decimal('18.5'), Decimal('-139.8'), 70, 0, 'H1', 'Eastern Pacific', 1.70294)
(2006, 1976, 7, 22, '0000Z', 312, 'DIANA', Decimal('18.6'), Decimal('-139.8'), 30, 0, 'TD', 'Eastern Pacific', 1.6)
(2007, 1978, 8, 26, '1200Z', 342, 'KRISTY', Decimal('21.4'), Decimal('-140.2'), 45, 0, 'TS', 'Eastern Pacific', 1.30384)
(2008, 1980, 9, 24, '1800Z', 371, 'KAY', Decimal('20.5'), Decimal('-140.2'), 75, 0, 'H1', 'Eastern Pacific', 1.22066)
(2009, 1970, 8, 23, '0000Z', 223, 'MAGGIE', D

In [23]:
conn.close()